In [1]:
# 주피터노트북 목차 만들기
# 참고: https://mingchin.tistory.com/139

# pip install jupyter_contrib_nbextensions
# ! jupyter contrib nbextension install --user

# 목차
> ## 1. 신뢰도 가중 협업필터링
>> ### 1.1. 데이터를 읽어오고 정리하기
>> ### 1.2. CF_knn_bias_sig 설명

# 1. 신뢰도 가중 협업필터링
<요약>  
신뢰도 가중 협업필터링은, 평가 경향 고려(평점 편차) 협업 필터링을 활용한 알고리즘이다.  
즉, KNN 이웃 기반 + 평점 편차 + 신뢰도 가중이라고 말할 수 있다.  
평가 경향 고려 협업 필터링에서는 
유사도가 높은 상위 [Neighbor size]명의 movie_id에 대한 **평점의 평균 편차**를 이용해 **가중 평균**을 구한 후,  
user_id의 전체 평점 평균에 더해주어 예상 평점을 구했다.  
  
지금까지 이웃 범위의 기준은 유사도가 높은 것, 딱 하나만 존재했다.  
신뢰도 가중 협업필터링은 이웃 범위의 기준을 추가하여,  
원하는 user_id와 공통으로 평가한 영화의 개수가 [SIG_LEVEL]개 이상으로 정해준다.  
추가적으로,  
movie_id에 평점을 부여한 사람이 최소 [MIN_RATINGS]명 이상인 경우에만 예상 평점 계산을 해주고,  
[MIN_RATINGS]명 미만의 경우에는 user_id의 전체 평점 평균을 넣어주는 방식으로  
해당 계산식에 신뢰도를 높여준다
  
다시 말해, 신뢰도를 높이기 위해
- 이웃의 범위
    - 유사도가 높은 [Neighbor size]명  
    - 인자로 받는 user_id와 공통으로 평가한 영화의 개수가 [SIG_LEVEL]개 이상  
- 예상 평점 계산을 하는 경우
    - 이웃의 범위를 추스린 후, movie_id에 평점을 부여한 사람이 최소 [MIN_RATINGS]명 이상인 경우  
  
로 정해준다
      
      
<코드 요약>  

>- 데이터 읽어오고 정리(RMSE 함수와 score(RMSE 사용)함수)하기
>- 각 사용자의 평점 편차 구해주기
>- 인자로 받은 user_id가 다른 사용자와 공통으로 평점을 부여한 항목과  
각 사용자가 평점을 부여한 영화의 개수 구하기
>- def CF_knn_bias_sig(user_id, movie_id, neighbor_size)
>    - 전달받은 movie_id에 대해 점수를 부여한 사람의 값이 rating_matrix에 존재할 때
>        - movie_id에 대한 평점이 없는 사람 제거하기
>        - user_id와 공통으로 평가한 영화의 개수가 SIG_LEVEL보다 작은 사람 제거하기
>        - 이웃의 크기가 1 이상일 때
>            - movie_id에 대해 평가한 사람이 [MIN_RATINGS]명 보다 클 때  
(prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()  
prediction = prediction + rating_mean[user_id])  
>            (평균 편차를 이용해 '평점 편차'를 구한 후, user_id의 전체 평점 평균에 더해준다)
>            - movie_id에 대해 평가한 사람이 [MIN_RATINGS]명 이상일 때  
(prediction = rating_mean[user_id])(=user_id의 전체 평점 평균)
>        - 이웃의 크기가 0일때  
(prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()  
prediction = prediction + rating_mean[user_id])  
>            (Neighbor size가 없기 때문에 전체 데이터를 이용해 계산해준다)
>    - 전달받은 movie_id에 대해 점수를 부여한 사람의 값이 rating_matrix에 존재하지 않을 때  
(prediction = rating_mean[user_id])(=user_id의 전체 평점 평균)
>- 예상 평점값 도출 (prediction)  
  
 --------------------------------------------------  
    
<가정>  
  
코드에 대한 이해를 돕기 위해 예시로,  
user_id = 2, movie_id = 2, neighbor_size = 30, SIG_LEVEL = 3, MIN_RATINGS =  2
값을 넣는다고 가정하고 코드 실행 

## 1.1. 데이터를 읽어오고 정리하기

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

In [4]:
users

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [5]:
movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [6]:
ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


#### - train셋과 test셋 분리하기
**train데이터로 알고리즘**을 제작 후, **test로 정확도를 측정**해야 정확한 검증을 할 수 있다  
따라서 데이터를 train셋과 test셋으로 분리해준다

In [7]:
# train, test 데이터 분리
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [8]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

#### - full matrix 제작과 코사인유사도 구하기
train용으로 분리가 완료된 x_train 데이터를 full matrix로 구현해준다(=rating_matrix)  
rating_matrix를 이용해 코사인 유사도를 구해준다(=user_similarity)

- **rating_matrix (x_train의 full matrix)**

In [9]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- **user_similarity (rating_matrix 활용해 구한 코사인유사도)**

In [10]:
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.077467,0.019205,0.041764,0.327722,0.339442,0.337730,0.195654,0.061163,0.245077,...,0.331247,0.099020,0.228108,0.134846,0.128228,0.085601,0.261737,0.132658,0.120340,0.298215
2,0.077467,1.000000,0.105362,0.150451,0.037438,0.154534,0.090013,0.068664,0.155715,0.117861,...,0.070736,0.240350,0.264485,0.321692,0.265980,0.194480,0.128473,0.135799,0.114154,0.060121
3,0.019205,0.105362,1.000000,0.310561,0.000000,0.052111,0.064324,0.059024,0.059808,0.048634,...,0.009645,0.000000,0.048084,0.048438,0.104583,0.036160,0.134245,0.050214,0.134790,0.015370
4,0.041764,0.150451,0.310561,1.000000,0.012908,0.011458,0.063566,0.156457,0.000000,0.037429,...,0.039022,0.000000,0.105375,0.176919,0.126255,0.040638,0.150687,0.139671,0.149319,0.031093
5,0.327722,0.037438,0.000000,0.012908,1.000000,0.199017,0.266480,0.225398,0.037633,0.113360,...,0.291910,0.047490,0.080681,0.055538,0.092553,0.080899,0.204377,0.109005,0.183357,0.203743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.085601,0.194480,0.036160,0.040638,0.080899,0.073518,0.106511,0.094348,0.000000,0.030914,...,0.067924,0.292145,0.163836,0.166962,0.238188,1.000000,0.046702,0.147718,0.058362,0.073076
940,0.261737,0.128473,0.134245,0.150687,0.204377,0.292324,0.229891,0.211737,0.076471,0.251317,...,0.298491,0.057717,0.136622,0.109477,0.094114,0.046702,1.000000,0.098494,0.238743,0.158560
941,0.132658,0.135799,0.050214,0.139671,0.109005,0.114853,0.053543,0.124717,0.083292,0.065964,...,0.021491,0.132770,0.288687,0.200871,0.248069,0.147718,0.098494,1.000000,0.064359,0.028541


#### - 각 사용자의 아이템별 평점 편차 구하기
각 사용자별로 평점의 부여하는 성향이 다르다.  
그렇기 때문에 평점 자체를 이용해 예상 평점을 구하는 것은  
전달 과정에서 의미가 변질될 리스크가 존재한다.  
따라서 리스크를 최소화 시키기 위해 평점 자체가 아닌, 각 항목별 평균 편차를 사용해준다

In [11]:
# rating_matrix의 각 user_id별 평균을 구해준다
rating_mean = rating_matrix.mean(axis=1)
rating_mean

user_id
1      3.671569
2      3.617021
3      2.725000
4      4.277778
5      2.969466
         ...   
939    4.216216
940    3.487500
941    4.000000
942    4.237288
943    3.484127
Length: 943, dtype: float64

In [12]:
# full matrix인 rating_matrix에서 rating_mean을 빼준다.
# 이것을 평점편차라고 부른다
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,-0.671569,0.328431,-0.671569,-0.671569,1.328431,0.328431,-2.671569,1.328431,-0.671569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.382979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.617021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.030534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.487500,NaN,NaN,NaN,1.512500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### - 공통항목과 평점을 부여한 항목 구해주기
rating_matrix를 활용해 user_id와 공통으로 평점을 부여한 영화의 개수를 구해준다.  
추가적으로 각각의 사용자들이 평가를 한 영화의 개수를 구해준다.  
  
이는 rating_matrix와 rating_matrix를 전치시킨 것의 내적으로 구할 수 있는데  
내적값의 각 셀은 각각의 사용자들이 공통으로 평가한 영화의 개수가 되고,  
대각선 값은 해당 사용자들이 평가한 영화의 총 개수가 된다.

In [13]:
# rating_matrix와 전치된 rating_martix를 내적 연산해준다.
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)
counts

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,204.0,7.0,3.0,3.0,50.0,56.0,85.0,16.0,3.0,39.0,...,54.0,9.0,31.0,10.0,19.0,8.0,35.0,7.0,13.0,51.0
2,7.0,47.0,7.0,5.0,2.0,15.0,12.0,4.0,4.0,9.0,...,6.0,8.0,18.0,11.0,16.0,7.0,8.0,5.0,6.0,4.0
3,3.0,7.0,40.0,9.0,0.0,5.0,9.0,5.0,1.0,4.0,...,1.0,0.0,5.0,3.0,7.0,2.0,9.0,2.0,6.0,1.0
4,3.0,5.0,9.0,18.0,1.0,1.0,6.0,5.0,0.0,2.0,...,2.0,0.0,5.0,4.0,5.0,1.0,6.0,3.0,6.0,2.0
5,50.0,2.0,0.0,1.0,131.0,27.0,56.0,15.0,2.0,15.0,...,38.0,3.0,10.0,3.0,10.0,6.0,17.0,4.0,16.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,8.0,7.0,2.0,1.0,6.0,7.0,12.0,3.0,0.0,2.0,...,5.0,8.0,10.0,6.0,12.0,37.0,2.0,4.0,3.0,4.0
940,35.0,8.0,9.0,6.0,17.0,32.0,37.0,11.0,3.0,29.0,...,31.0,3.0,14.0,5.0,8.0,2.0,80.0,4.0,15.0,14.0
941,7.0,5.0,2.0,3.0,4.0,7.0,5.0,5.0,1.0,4.0,...,1.0,3.0,12.0,4.0,9.0,4.0,4.0,17.0,2.0,1.0


## 1.2. CF_knn_bias_sig 설명
기존의 평가 경향 고려(평점 편차) 협업필터링에 추가적으로  
- 이웃 범위의 조건을 추가(user_id와 공통으로 평점을 부여한 영화의 개수가 [SIG_LEVEL]개 이상)  
- movie_id에 대해 평가를 한 사용자가 최소 [MIN_RATINGS]명 이상  
  
이라는 조건을 추가해주었다

- user_id = 2, movie_id = 2, neighbor_size = 30, SIG_LEVEL = 3, MIN_RATINGS = 2 이라고 가정한다

In [14]:
user_id = 2
movie_id = 2
neighbor_size = 30
SIG_LEVEL = 3
MIN_RATINGS = 2 

#### - 전달받은 movie_id가 rating_matrix에 존재하는지 확인이 필요하다
rating_matrix는 x_train을 활용해 만든 데이터 셋이다.  
따라서 탐색을 원하는 movie_id에 대한 정보가 들어있지 않을 수 있다  
만약 movie_id가 rating_matrix에 존재하지 않는다면(Null), 평균 점수로 정한 3.0점을 평점으로 부여한다
#### - 전달받은movie_id가 rating_matrix에 존재한다면, user_id에 대한 코사인 유사도와 movie_id에 대한 평점들을 받아온다
user_id와 다른 사용자의 코사인 유사도가 담긴 user_similarity를 추출해 sim_scores에 담아준다  
movie_id에 대해 다른 사용자들이 부여한 평점을 rating_matrix에서 추출해 movie_ratings에 담아준다

#### - movie_id에 대해 평가하지 않은 사용자의 정보는 사용 데이터에서 제외시켜준다
movie_id에 평점을 부여하지 않은 사용자들을  
movie_id의 평점 모음인 movie_ratings와 user_id와 다른 사용자간의 유사도 모음인 sim_scores에서 제외시켜준다  
  
#### - user_id와 공통으로 평가한 영화의 개수가 [SIG_LEVEL]개 보다 작은 경우는 사용 데이터에서 제외시켜준다

현재는 user_id = 2, movie_id = 2, neighbor_size = 30 이라고 가정하기 때문에  
movie_id에 대해 평점을 부여한 사람들의 정보(평점, 유사도)만을 받아준다 -> 'movie_ratings', 'sim_scores'  
그 후 공통 평가 영화 개수가 [SIG_LEVEL]개 보다 작은 경우 'movie_ratings', 'sim_scores'에서 제외시켜준다

In [15]:
if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    no_rating = movie_ratings.isnull()
    common_counts = counts[user_id]
    low_significance = common_counts < SIG_LEVEL
    none_rating_idx = movie_ratings[no_rating | low_significance].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

- movie_ratings (movie_id에 대한 평점이 Null이 아닌 사람들의 **평점 모음**)

In [16]:
movie_ratings

user_id
1     -0.671569
13    -0.111111
22    -1.260417
30    -0.812500
42     1.299270
         ...   
892   -0.071006
916   -0.399160
924   -0.770492
934    0.274809
943    1.515873
Name: 2, Length: 91, dtype: float64

- sim_scores (movie_id에 대한 평점이 Null이 아닌 사람들의 **유사도 모음**)

In [17]:
sim_scores

user_id
1      0.077467
13     0.134779
22     0.060870
30     0.222819
42     0.086937
         ...   
892    0.098754
916    0.079483
924    0.166638
934    0.070736
943    0.060121
Name: 2, Length: 91, dtype: float64

#### - Neighbor size(이웃의 크기)가 0인 경우와 1 이상인 경우
여기서 Neighbor size는 후에 'user_id와 유사도가 높은 [Neighbor size]개'에 사용된다 
만약 Neighbor size가 0이라면 유사도가 높은 0개의 유사도를 구하게 되고, 그것은 의미없는 값이다  
  
- 따라서 **Neighbor size가 0**일땐 범위를 따로 정하지 않고,  
사용자 범위를 전체로 삼아  
movie_id에 대한 평점 편차(movei_ratings)에 유사도를(sim_scores)를 가중치 삼아 '평점 편차의 가중평균'을 구해준다.  
그 후 user_id의 전체 평점 평균(rating_mean[user_id]) + '평점 편차의 가중평균'을 계산해 예상 평점에 넣어준다.
  
  
- 반대로 **Neighbor size가 1이상**이라면,  
sim_scores 관련 if문으로 이동하게 된다
  
  
#### - Neighbor size가 1 이상이면 그 후 sim_scores관련 if문이 등장한다
sim_scores는 평점을 부여한 사람들의 user_id에 대한 유사도 모음  
신뢰도를 더 높이기 위해  
sim_scores가 [MIN_RATINGS]이하인 경우와 초과인 경우를 나눠서 계산해준다.  
  
- **sim_score가 [MIN_RATINGS]개 이하**인 경우는 강제로  
user_id의 전체 평점 평균을 예상 평점값으로 넣어준다
  
  
- 반대로 **sim_scores가 [MIN_RATINGS]이상**인 경우에는  
사용자와 유사도가 높은 [Neighbor size]개를 범위로 잡는다  
movie_id에 대한 평점 편차(movei_ratings)에 유사도를(sim_scores)를 가중치 삼아 '평점 편차의 가중평균'을 구해준다.  
그 후 user_id의 전체 평점 평균(rating_mean[user_id]) + '평점 편차의 가중평균'을 계산해 예상 평점에 넣어준다.

현재는 user_id = 2, movie_id = 2, neighbor_size = 30 이라고 가정하고 있다  
Neighbor size는 0보다 크고, sim_scores 역시 [MIN_RATINGS]보다 큰 상태이다  
따라서 사용자와 **유사도가 높은 [Neighbor size]명**의 **'평점 편차의 가중평균'**을 구한 후  
user_id의 전체 평점 평균에 더해줘서 예상 평점을 구해준다

In [18]:
if len(sim_scores) > MIN_RATINGS:
    neighbor_size = min(neighbor_size, len(sim_scores))
    sim_scores = np.array(sim_scores)
    movie_ratings = np.array(movie_ratings)
    user_idx = np.argsort(sim_scores)
    sim_scores = sim_scores[user_idx][-neighbor_size:]
    movie_ratings = movie_ratings[user_idx][-neighbor_size:]
    prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    print('평점편차:',prediction)
    prediction = prediction + rating_mean[user_id]

prediction

평점편차: -0.347010937662819


3.270010338932926

코드 원본

In [19]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id]
        movie_ratings = rating_bias[movie_id]
        no_rating = movie_ratings.isnull()
        common_counts = counts[user_id]
        low_significance = common_counts < SIG_LEVEL
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        if neighbor_size == 0:
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        else:        
            if len(sim_scores) > MIN_RATINGS:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction